In [35]:
import cv2 as cv
import tensorflow as tf
import numpy as np
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay

In [ ]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('models/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("models/emotion_model.h5")

In [18]:
face_cascade = cv.CascadeClassifier('cascade/haarcascade_frontalface_alt2.xml')
img = cv.imread('R.jpeg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

In [20]:
for (x, y, w, h) in faces: 
    cv.rectangle(img, (x, y), (x+w, y+h),  
                  (0, 0, 255), 2) 
      
    faces = img[y:y + h, x:x + w] 
    cv.imshow("face",faces) 
    cv.imwrite('face.jpg', faces) 
  
cv.imshow('img', img) 
cv.waitKey()

-1

In [26]:
gray_face=cv.cvtColor(faces, cv.COLOR_BGR2GRAY)
resized=cv.resize(gray_face, (48,48))
print(resized.shape)

(48, 48)


In [22]:
cv.imshow('img', resized)
cv.waitKey()

-1

In [65]:
image_tensor = tf.convert_to_tensor(resized, dtype=tf.float32)

# Add dimension to match with input mode 
image_tensor = tf.expand_dims(image_tensor, 2)
image_tensor = tf.expand_dims(image_tensor, 0)
image_tensor=image_tensor/255
emotion_dict[emotion_model.predict(image_tensor).argmax()]

1/1 [==============================] - 0s 21ms/step


'Happy'